In [23]:
from datetime import datetime
from sqlalchemy import create_engine
import pymysql

In [1]:

import mysql.connector
import pandas as pd

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    # Conexión a la base de datos
    db_connection = mysql.connector.connect(
        host="127.0.0.1",
        user="root",  # Cambia por tu usuario de MySQL
        password="11jablum11",  # Cambia por tu contraseña de MySQL
        database="yfinance"
    )
    cursor = db_connection.cursor()



    # Consulta SQL para obtener el precio inicial y final en el periodo
    query = """
    SELECT fecha, precio_cierre
    FROM precios_historicos
    WHERE id_empresa = (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s)
      AND fecha BETWEEN %s AND %s
    ORDER BY fecha ASC;
    """
    
    cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
    precios = cursor.fetchall()
    
    # Cerrar conexión
    cursor.close()
    db_connection.close()

    # Verifica si se obtuvieron precios en el rango de fechas
    if len(precios) < 2:
        st.warning("No se encontraron datos suficientes en el rango de fechas seleccionado.")
        return None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])

### Paso 2: Crear la Interfaz de Streamlit



In [38]:
import pymysql

# Datos de conexión
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def calcular_roi(simbolo, fecha_compra, fecha_venta):
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta para el precio de compra (precio de cierre en la fecha de compra)
        query_compra = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_compra, (simbolo, fecha_compra))
        resultado_compra = cursor.fetchone()

        # Consulta para el precio de venta (precio de cierre en la fecha de venta)
        query_venta = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_venta, (simbolo, fecha_venta))
        resultado_venta = cursor.fetchone()

        # Verificar que se encontraron precios en ambas fechas
        if resultado_compra and resultado_venta:
            precio_compra = resultado_compra[0]
            precio_venta = resultado_venta[0]

            # Calcular ROI
            ganancia = precio_venta - precio_compra
            roi = (ganancia / precio_compra) * 100

            print(f"Empresa: {simbolo}")
            print(f"Precio de compra ({fecha_compra}): {precio_compra}")
            print(f"Precio de venta ({fecha_venta}): {precio_venta}")
            print(f"ROI entre {fecha_compra} y {fecha_venta}: {round(roi, 2)}%")
        else:
            if not resultado_compra:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de compra {fecha_compra}.")
            if not resultado_venta:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de venta {fecha_venta}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        # Cerrar la conexión
        if connection:
            connection.close()

# Ejemplo de uso de la función
calcular_roi("AAPL", "2020-01-02", "2020-12-31")  # Cambia "AAPL" y las fechas por los valores que necesitas verificar


Empresa: AAPL
Precio de compra (2020-01-02): 123.615
Precio de venta (2020-12-31): 124.507
ROI entre 2020-01-02 y 2020-12-31: 0.72%
